<a href="https://colab.research.google.com/github/Visors/d2l-zh-2/blob/main/tensorflow/chapter_deep-learning-computation/parameters.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 参数管理

在选择了架构并设置了超参数后，我们就进入了训练阶段。
此时，我们的目标是找到使损失函数最小化的模型参数值。
经过训练后，我们将需要使用这些参数来做出未来的预测。
此外，有时我们希望提取参数，以便在其他环境中复用它们，
将模型保存下来，以便它可以在其他软件中执行，
或者为了获得科学的理解而进行检查。

之前的介绍中，我们只依靠深度学习框架来完成训练的工作，
而忽略了操作参数的具体细节。
本节，我们将介绍以下内容：

* 访问参数，用于调试、诊断和可视化；
* 参数初始化；
* 在不同模型组件间共享参数。

(**我们首先看一下具有单隐藏层的多层感知机。**)


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import tensorflow as tf

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4, activation=tf.nn.relu),
    tf.keras.layers.Dense(1),
])

X = tf.random.uniform((2, 4))
net(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[ 0.        ],
       [-0.44521675]], dtype=float32)>

## [**参数访问**]

我们从已有模型中访问参数。
当通过`Sequential`类定义模型时，
我们可以通过索引来访问模型的任意层。
这就像模型是一个列表一样，每层的参数都在其属性中。
如下所示，我们可以检查第二个全连接层的参数。


In [3]:
print(net.layers[2].weights)

[<Variable path=sequential/dense_1/kernel, shape=(4, 1), dtype=float32, value=[[-0.3750403 ]
 [ 1.0805106 ]
 [-0.81241035]
 [-0.24212623]]>, <Variable path=sequential/dense_1/bias, shape=(1,), dtype=float32, value=[0.]>]


输出的结果告诉我们一些重要的事情：
首先，这个全连接层包含两个参数，分别是该层的权重和偏置。
两者都存储为单精度浮点数（float32）。
注意，参数名称允许唯一标识每个参数，即使在包含数百个层的网络中也是如此。

### [**目标参数**]

注意，每个参数都表示为参数类的一个实例。
要对参数执行任何操作，首先我们需要访问底层的数值。
有几种方法可以做到这一点。有些比较简单，而另一些则比较通用。
下面的代码从第二个全连接层（即第三个神经网络层）提取偏置，
提取后返回的是一个参数类实例，并进一步访问该参数的值。


In [4]:
print(type(net.layers[2].weights[1]))
print(net.layers[2].weights[1])
print(tf.convert_to_tensor(net.layers[2].weights[1]))

<class 'keras.src.backend.Variable'>
<Variable path=sequential/dense_1/bias, shape=(1,), dtype=float32, value=[0.]>
tf.Tensor([0.], shape=(1,), dtype=float32)


### [**一次性访问所有参数**]

当我们需要对所有参数执行操作时，逐个访问它们可能会很麻烦。
当我们处理更复杂的块（例如，嵌套块）时，情况可能会变得特别复杂，
因为我们需要递归整个树来提取每个子块的参数。
下面，我们将通过演示来比较访问第一个全连接层的参数和访问所有层。


In [5]:
print(net.layers[1].weights)
print(net.get_weights())

[<Variable path=sequential/dense/kernel, shape=(4, 4), dtype=float32, value=[[ 0.04279369 -0.7751032   0.583455   -0.49887913]
 [ 0.15470976 -0.3488673  -0.74298    -0.6117219 ]
 [-0.35123974  0.4011336  -0.579068    0.5781167 ]
 [ 0.23326808 -0.2689491   0.3871935  -0.400287  ]]>, <Variable path=sequential/dense/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>]
[array([[ 0.04279369, -0.7751032 ,  0.583455  , -0.49887913],
       [ 0.15470976, -0.3488673 , -0.74298   , -0.6117219 ],
       [-0.35123974,  0.4011336 , -0.579068  ,  0.5781167 ],
       [ 0.23326808, -0.2689491 ,  0.3871935 , -0.400287  ]],
      dtype=float32), array([0., 0., 0., 0.], dtype=float32), array([[-0.3750403 ],
       [ 1.0805106 ],
       [-0.81241035],
       [-0.24212623]], dtype=float32), array([0.], dtype=float32)]


这为我们提供了另一种访问网络参数的方式，如下所示。


In [6]:
net.get_weights()[1]

array([0., 0., 0., 0.], dtype=float32)

### [**从嵌套块收集参数**]

让我们看看，如果我们将多个块相互嵌套，参数命名约定是如何工作的。
我们首先定义一个生成块的函数（可以说是“块工厂”），然后将这些块组合到更大的块中。


In [7]:
def block1(name):
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(4, activation=tf.nn.relu)],
        name=name)

def block2():
    net = tf.keras.Sequential()
    for i in range(4):
        # 在这里嵌套
        net.add(block1(name=f'block-{i}'))
    return net

rgnet = tf.keras.Sequential()
rgnet.add(block2())
rgnet.add(tf.keras.layers.Dense(1))
rgnet(X)

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[0.],
       [0.]], dtype=float32)>

[**设计了网络后，我们看看它是如何工作的。**]


In [8]:
print(rgnet.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ sequential_2 (Sequential)            │ (2, 4)                      │              80 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_6 (Dense)                      │ (2, 1)                      │               5 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 85 (340.00 B)

 Trainable params: 85 (340.00 B)

 Non-trainable params: 0 (0.00 B)

None


因为层是分层嵌套的，所以我们也可以像通过嵌套列表索引一样访问它们。
下面，我们访问第一个主要的块中、第二个子块的第一层的偏置项。


In [9]:
rgnet.layers[0].layers[1].layers[1].weights[1]

<Variable path=sequential_1/sequential_2/block-1/dense_3/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>

## 参数初始化

知道了如何访问参数后，现在我们看看如何正确地初始化参数。
我们在 :numref:`sec_numerical_stability`中讨论了良好初始化的必要性。
深度学习框架提供默认随机初始化，
也允许我们创建自定义初始化方法，
满足我们通过其他规则实现初始化权重。


默认情况下，Keras会根据一个范围均匀地初始化权重矩阵，
这个范围是根据输入和输出维度计算出的。
偏置参数设置为0。
TensorFlow在根模块和`keras.initializers`模块中提供了各种初始化方法。


### [**内置初始化**]

让我们首先调用内置的初始化器。
下面的代码将所有权重参数初始化为标准差为0.01的高斯随机变量，
且将偏置参数设置为0。


In [10]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.random_normal_initializer(mean=0, stddev=0.01),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1)])

net(X)
net.weights[0], net.weights[1]

(<Variable path=sequential_3/dense_7/kernel, shape=(4, 4), dtype=float32, value=[[-0.00440194 -0.0129707  -0.00981739 -0.02151016]
  [-0.00137896 -0.00761776  0.00632287 -0.01950469]
  [ 0.00486974  0.00259039  0.00521421  0.00774102]
  [-0.0054437   0.00411853 -0.00939435 -0.004771  ]]>,
 <Variable path=sequential_3/dense_7/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>)

我们还可以将所有参数初始化为给定的常数，比如初始化为1。


In [11]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4, activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.Constant(1),
        bias_initializer=tf.zeros_initializer()),
    tf.keras.layers.Dense(1),
])

net(X)
net.weights[0], net.weights[1]

(<Variable path=sequential_4/dense_9/kernel, shape=(4, 4), dtype=float32, value=[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]>,
 <Variable path=sequential_4/dense_9/bias, shape=(4,), dtype=float32, value=[0. 0. 0. 0.]>)

我们还可以[**对某些块应用不同的初始化方法**]。
例如，下面我们使用Xavier初始化方法初始化第一个神经网络层，
~~然后将第三个神经网络层初始化为常量值42。~~

（mio按 应该是将第二个神经网络层初始化为常量值42，且代码也需要一并修改。）

In [12]:
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=tf.keras.initializers.GlorotUniform()),
    # tf.keras.layers.Dense(
    #     1, kernel_initializer=tf.keras.initializers.Constant(1)),
    tf.keras.layers.Dense(
        1, kernel_initializer=tf.keras.initializers.Constant(42)),
])

net(X)
print(net.layers[1].weights[0])
print(net.layers[2].weights[0])

<Variable path=sequential_5/dense_11/kernel, shape=(4, 4), dtype=float32, value=[[-0.42683843  0.0887692   0.06075281 -0.22332442]
 [-0.7090687   0.06450164 -0.13744998  0.28668565]
 [ 0.5665093   0.70073515  0.05948591  0.11898685]
 [-0.5601326   0.71856004  0.32070142  0.17591804]]>
<Variable path=sequential_5/dense_12/kernel, shape=(4, 1), dtype=float32, value=[[42.]
 [42.]
 [42.]
 [42.]]>


### [**自定义初始化**]

有时，深度学习框架没有提供我们需要的初始化方法。
在下面的例子中，我们使用以下的分布为任意权重参数$w$定义初始化方法：

$$
\begin{aligned}
    w \sim \begin{cases}
        U(5, 10) & \text{ 可能性 } \frac{1}{4} \\
            0    & \text{ 可能性 } \frac{1}{2} \\
        U(-10, -5) & \text{ 可能性 } \frac{1}{4}
    \end{cases}
\end{aligned}
$$


在这里，我们定义了一个`Initializer`的子类，
并实现了`__call__`函数。
该函数返回给定形状和数据类型的所需张量。


In [13]:
class MyInit(tf.keras.initializers.Initializer):
    def __call__(self, shape, dtype=None):
        data=tf.random.uniform(shape, -10, 10, dtype=dtype)
        factor=(tf.abs(data) >= 5)
        factor=tf.cast(factor, tf.float32)
        return data * factor

net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(
        4,
        activation=tf.nn.relu,
        kernel_initializer=MyInit()),
    tf.keras.layers.Dense(1),
])

net(X)
print(net.layers[1].weights[0])

<Variable path=sequential_6/dense_13/kernel, shape=(4, 4), dtype=float32, value=[[-0.         9.91086   -7.180207  -0.       ]
 [ 0.        -0.         0.         7.3522377]
 [-7.8321767  7.222006  -5.4717326 -6.2572575]
 [ 0.        -7.8836274  9.820463  -0.       ]]>


注意，我们始终可以直接设置参数。


In [14]:
net.layers[1].weights[0][:].assign(net.layers[1].weights[0] + 1)
net.layers[1].weights[0][0, 0].assign(42)
net.layers[1].weights[0]

<Variable path=sequential_6/dense_13/kernel, shape=(4, 4), dtype=float32, value=[[42.        10.91086   -6.180207   1.       ]
 [ 1.         1.         1.         8.352238 ]
 [-6.8321767  8.222006  -4.4717326 -5.2572575]
 [ 1.        -6.8836274 10.820463   1.       ]]>

## [**参数绑定**]

有时我们希望在多个层间共享参数：
我们可以定义一个稠密层，然后使用它的参数来设置另一个层的参数。


In [17]:
# tf.keras的表现有点不同。它会自动删除重复层
shared = tf.keras.layers.Dense(4, activation=tf.nn.relu)
net = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    shared,
    shared,
    tf.keras.layers.Dense(1),
])

net(X)
# 检查参数是否不同
# print(len(net.layers) == 3)
print(tf.__version__)
print(len(net.layers))  # tf 2.18.0 下似乎不会自动删除重复层了

2.18.0
4


## 小结

* 我们有几种方法可以访问、初始化和绑定模型参数。
* 我们可以使用自定义初始化方法。

## 练习

1. 使用 :numref:`sec_model_construction` 中定义的`FancyMLP`模型，访问各个层的参数。
1. 查看初始化模块文档以了解不同的初始化方法。
1. 构建包含共享参数层的多层感知机并对其进行训练。在训练过程中，观察模型各层的参数和梯度。
1. 为什么共享参数是个好主意？


[Discussions](https://discuss.d2l.ai/t/1830)
